In [17]:
from traitlets import (HasTraits, Float, Int, List, Instance)
from traittypes import DataFrame
from collections import deque
import pandas as pd
import numpy as np

In [ ]:
# Welcome to today!! It's going to be a wonderful day. Let's get out a new sheet of paper and do our best to understand 
# this WOFOST Leaf Dynamics model and how it works

In [49]:
class Foo(HasTraits) :
    attr = Int(-10)
    
    #so you can assign the attributes values in the __init__ method or you can just set up the 
    #different attributes and then assign them values after you create a Foo object
    def __init__(self, attr) :
        self.attr = attr

In [ ]:
class Model :
    def __init__(self) :
        self.all_data = pd.DataFrame(columns=states_and_rates) # or columns=pantry.pantry_dict.keys()

    def record_all(self, pantry) :
        todays_data = pd.DataFrame(pantry.pantry_dict) 
        self.all_data = pd.concat([self.all_data, todays_data], ignore_index=True, axis=0)
        
    def run(self) :
        pass
    def potential_production(self) :
        pass

In [5]:
class Pantry :
    def __init__(self) :
        self.pantry_dict = dict()
        
    def publish(self, dict_of_variables) :
        self.pantry_dict.update(dict_of_variables)

In [118]:
class ParameterTemplate(HasTraits) :
    def __init__(self, **kwargs) :
        for key, value in kwargs.items() :
            if key in self.__dir__() :
                self.__setattr__(key, value) 

In [7]:
class StatesTemplate(HasTraits) :
    def __init__(self, pantry, **kwargs) :
        pantry.publish(kwargs) # record the initial states
        for key, value in kwargs.items() :
            self.__setattr__(key, value) 
           
    def publish(self, pantry) :
        pantry.publish(self.__dict__['_trait_values'])

In [8]:
class RatesTemplate(HasTraits) :
    def __init__(self, pantry) :
        pantry.publish(self.__dict__['_trait_values'])
    
    def publish(self, pantry) :
        pantry.publish(self.__dict__['_trait_values'])

In [8]:
def get_val(df, DVS) :
    return df.iloc[np.where(df.iloc[:,0] <= DVS)[0][-1],1]

In [130]:
class Leaf_Dynamics : # could do one based off of CSDM 
    
    class Parameters(ParameterTemplate) :
        PLANT_INITIAL_WEIGHT = Float()
        BASE_TEMP = Int()
        AGING_LIFESPAN35 = Int()
        LV_PART = DataFrame()
        SLA_FUNC = DataFrame() # as a step function of DVS
        MAX_REL_GROWTH = Float()
    
    class StateVariables(StatesTemplate) :
        LAI = Float()
        SLA = Float() 
        LEAVES = Instance(deque) # queue of leaf objects that come on and die off
        LV_TOTAL_WEIGHT = Float()
        LV_DEAD_WEIGHT = Float()
        LV_LIVE_WEIGHT = Float()
        
    class RateVariables(RatesTemplate) :
        H20_O2_SENES = Float()
        SHADING_SENES = Float()
        AGING_SENES = Float()
        TOTAL_SENES = Float()
        LAIEXP = Float()
        NEW_LV_GROWTH = Float()
        LAIBM = Float()
        LAI_GRW = Float()
        CUR_SLA = Float()
            
    class Leaf :
        def __init__(self, biomass, SLA) :
            self.weight = biomass
            self.SLA = SLA
            self.age = 0.0
            
        def update_age_linear(self, temp) :     
            self.age += temp 
            
        def update_age_asym(self, params, temp) : # param: params - a Parameters obj
            self.age += np.amax(0, (temp - params.BASE_TEMP)/(35 - temp))
    
    def __init__(self, pantry, **params) :
        self.pantry = pantry
        self.params = self.Parameters(**params)
        
        # determine initial states based off parameters
        p = self.params
        LV_EMG_WEIGHT = p.PLANT_INTIAL_WEIGHT * get_val(p.LV_PART, DVS=0)
        SLA = get_val(p.SLA_FUNC, DVS=0)
        LAI = LV_EMG_WEIGHT * SLA
        LV_TOTAL_WEIGHT = LV_EMG_WEIGHT
        LV_DEAD_WEIGHT = 0
        LV_LIVE_WEIGHT = LV_EMG_WEIGHT
        
        first_leaf = Leaf(LV_TOTAL_WEIGHT, SLA)
        LEAVES = deque([first_leaf])
        
        self.rates = RateVariables()
        self.states = StateVariables(LAI=LAI, SLA=SLA, 
                                     LV_TOTAL_WEIGHT=LV_TOTAL_WEIGHT, LEAVES=LEAVES, 
                                     LV_DEAD_WEIGHT=LV_DEAD_WEIGHT,
                                     LV_LIVE_WEIGHT=LV_LIVE_WEIGHT)
        
    def calc_rates(self) :
        p = self.params
        pan = self.pantry
        s = self.states
        r = self.rates
        
        # death from water/oxygen stress
        r.H20_O2_SENES = p.H20_02_SENES_MAX * pan.TRANS_REDUCT_FACT * s.LV_TOTAL_WEIGHT
        # TRANS_REDUCT_FACT needs to be between 0 and 1 - what percent of max death
        
        # death from shading
        r.SHADING_SENES = 0
        ### need to come up with my own funct perhaps ### 
        
        # death from aging
        for leaf in s.LEAVES :
            if leaf.age > p.AGING_LIFESPAN35 :
                r.AGING_SENES += leaf.weight
        r.TOTAL_SENES = np.amax(r.H20_O2_SENES, r.SHADING_SENES, r.AGING_SENES)
    
        # LAI growth
        r.CUR_SLA = get_val(p.SLA_FUNC, pan.DVS)
        r.NEW_LV_GROWTH = pan.NEW_GROWTH * get_val(p.LV_PART, pan.DVS)
        r.GLAIEXP = r.LAIEXP * p.MAX_REL_GROWTH * (pan.temp_today - p.BASE_TEMP) # exponentially limited
        r.LAIBM = r.NEW_LV_GROWTH * r.CUR_SLA # "source limited" off of biomass
        r.LAI_GRW = np.amin(r.LAIEXP, r.LAIBM)
        
    def update_state_vars(self) :
        p = self.params
        pan = self.pantry
        s = self.states
        r = self.rates
        
        # some leaves die off
        for leaf in reversed(LEAVES) :
            if r.TOTAL_SENES > leaf.weight :
                r.TOTAL_SENES -= leaf.weight
                LEAVES.pop()
            else :
                leaf.weight -= r.TOTAL_SENES
                break
        
        # the leaves get older
        for leaf in LEAVES :
            leaf.update_age_asym(pan.todays_temp)
        
        # a new leaf is born
        LEAVES.appendleft(Leaf(r.NEW_LV_GROWTH, r.CUR_SLA))
        
        s.LAI = np.sum([leaf.weight * leaf.SLA for leaf in s.LEAVES])
        s.LAIEXP += r.GLAIEX
        
        # update weights
        s.LV_LIVE_WEIGHT += r.NEW_LV_GROWTH - r.TOTAL_SENES
        s.LV_DEAD_WEIGHT += r.TOTAL_SENES
        s.LV_TOTAL_WEIGHT += r.NEW_LV_GROWTH

In [132]:
p = Pantry()

In [87]:
class Parameters(ParameterTemplate) :
        INITIAL_WEIGHT = Float()
        INTIAL_LAI = Float()
        SLA = DataFrame()

In [ ]:
class Emergence_Model :
    pass

In [ ]:
# create object with parameters
# run simulation, have the results stored in a df, also have a summary report
# visualize the results

In [134]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime 
import numpy as np
from collections import deque

In [2]:
url = 'http://agebb.missouri.edu/weather/history/report.asp?station_prefix=bwk&start_month=1&end_month=2&start_day=1&end_day=15&start_year=2020&end_year=2020&period_type=1&convert=1&field_elements=70&field_elements=3&field_elements=51&field_elements=73'
headers = {'User-Agent':'Mozilla/5.0'}
response = requests.get(url, headers = headers)

In [3]:
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('pre')
data = table.contents[0].split()

In [4]:
column_names = ['Date', 'Total Precip', 'Max Temp', 'Min Temp', 'Total Solar Rad']
weather_data = pd.DataFrame(columns=column_names)

In [5]:
i = 18
while i < len(data) :
    newrow = data[i:i+7]    
    datestring = newrow[0] + '-' + newrow[1] + '-' + newrow[2]
    dateobj = datetime.datetime.strptime(datestring, '%m-%d-%Y').date()
    weather_data = weather_data.append({'Date': dateobj, 
                    'Total Precip': newrow[3], 
                    'Max Temp': newrow[4],
                    'Min Temp': newrow[5],
                    'Total Solar Rad': newrow[6]}, ignore_index=True)
    i += 7
weather_data['Date'] = pd.to_datetime(weather_data['Date'])
weather_data.set_index('Date', inplace = True)

In [18]:
crop_params = {
    'N stuff': 25,
    'k': 0.6,
    'LAI by stage':[[0, .003], [.5, .009], [1, .006], [1.5, .004], [2, .001]],
    'Max Increase of Roots per day': .012, # m d-1
    'Partitioning %s Roots': [
    [0.0, 0.60],
    [0.33, 0.58],
    [0.40, 0.55],
    [0.80, 0.10],
    [1.00, 0.00],
    [2.00, 0.0 ]],

    'Partitioning %s Leaves': [
    [0.0, 0.40],
    [0.33, 0.42],
    [0.40, 0.405],
    [0.80, 0.36],
    [1.00, 0.10],
    [1.01, 0.00],
    [2.00, 0.00]],

    'Partitioning %s Stem': [
    [0.0, 0.00],
    [0.33, 0.00],
    [0.40, 0.045],
    [0.80, 0.54],
    [1.00, 0.90],
    [1.01, 0.25],
    [2.00, 0.00]],

    'Partitioning %s Storage Organs': [
    [0.0, 0.00],
    [0.33, 0.00],
    [0.40, 0.00],
    [0.80, 0.00],
    [1.00, 0.00],
    [1.01, 0.75],
    [2.00, 1.00]],
    
    'RUE': 2.8, # g / MJ ?
    'Temp Sum for Emergence': 0,
    'Temp Sum for Anthesis': 800,
    'Temp Sum for Maturity': 1030, # all in celcius
    'Planting Date': datetime.date(2006, 4, 5)
}

In [19]:
soil_params = {
    'Initial N': 200,
    'Initial SM': .3,
    'Water Retention': [[.3, 2],[.4, 3]], # step function of SM and pF
    'SM @ WP': .09,
    'SM @ FC': .3,
    'SM @ Sat': .5
}

In [20]:
weather_data = pd.read_excel('nl1.xlsx', header=0, skiprows=[0,1,2,3,4,5,6,7,8,9])
weather_data['DAY'] = pd.to_datetime(weather_data['DAY'], errors='coerce')
weather_data['DAY'] = weather_data['DAY'].dt.date
weather_data = weather_data.set_index('DAY')

In [21]:
management_params = {
    'Amount N': 20,
    'pF to Maintain': 2.2
}

In [125]:
class Leaf :
    def __init__(self, LAI, biomass) :
        self.LAI = LAI
        self.biomass = biomass
        self.temp_sum = 0
    def update(self, temp_sum) :
        self.temp_sum += temp_sum

In [172]:
class Model :
    def __init__(self, crop, soil, weather, management) :
        # read variables 
        self.N = crop['N stuff']
        self.k = crop['k']
        self.LAIRates = pd.DataFrame(crop['LAI by stage'])
        self.RGMAX = crop['Max Increase of Roots per day']
        self.RT = pd.DataFrame(crop['Partitioning %s Roots'])
        self.LV = pd.DataFrame(crop['Partitioning %s Leaves'])
        self.ST = pd.DataFrame(crop['Partitioning %s Stem'])
        self.SO = pd.DataFrame(crop['Partitioning %s Storage Organs'])
        self.RUE = crop['RUE']
        self.TSUMEM = crop['Temp Sum for Emergence']
        self.TSUM1 = crop['Temp Sum for Anthesis']
        self.TSUM2 = crop['Temp Sum for Maturity']
        self.DOS = crop['Planting Date'] # date of sowing
        self.today = crop['Planting Date']
        self.LAI = .14
        self.DVS = 0
        self.TAGP = 0
        self.three = 0
        self.deque_leaves = deque()
        
        self.roots_partition = 0.0
        self.leaves_partition = 0.0
        self.stem_partition = 0.0
        self.storage_partition = 0.0
        
        self.storage = 0
        self.stem = 0
        self.leaves = 0
        self.roots = 0
        
        self.IN = soil['Initial N']
        self.ISM = soil['Initial SM']
        self.WRC = pd.DataFrame(soil['Water Retention']) # Water Retention Curve
        self.WPSM = soil['SM @ WP']
        self.FCSM = soil['SM @ FC']
        self.SSM = soil['SM @ Sat']
        
        self.weather = weather
        
        self.management = management
        
        self.data = pd.DataFrame(columns=['Date', 'Development Stage', 'LAI', 'Biomass', 
                                          'Storage', 'Stem', 'Leaves', 'Roots'])
        
    def update(self) :
        self.today += datetime.timedelta(days=1)
        
        # update partitioning proportions
        self.roots_partition = self.RT.iloc[np.where(self.RT.iloc[:,0] <= self.DVS)[0][-1],1]
        self.leaves_partition = self.LV.iloc[np.where(self.LV.iloc[:,0] <= self.DVS)[0][-1],1]
        self.stem_partition = self.ST.iloc[np.where(self.ST.iloc[:,0] <= self.DVS)[0][-1],1]
        self.storage_partition = self.SO.iloc[np.where(self.SO.iloc[:,0] <= self.DVS)[0][-1],1]
        
        # 
    
    def radiation_intercepted(self) :
        total_radiation = float(self.weather.loc[self.today]['IRRAD']) 
        radiation_intercepted = total_radiation * (1 - np.exp(-1 * self.k * self.LAI))
        return ri
    
    
    def LAI_growth_rate(self) :
        # maybe it's a constant, derived empirically at harvest by regressing LAI against temperature sum
        # in WOFOST model seems to be exponential at first, then experiences leaf decay
        # when N is incorporated into the model, the LAI growth rate as a constant calculated at harvest
        # has to do with the amount of N in the soil
        
        # seems like the options are: some exp funct of development stage, a constant value (as a parameter, like in WOFOST)
        # that is plugged into an exp funct until it reaches some 'peak growth stage', something else
        
        specific_leaf_area_constant = 0.022
        return specific_leaf_area_constant
        
    
    def run(self) :
        print(self.RT)
        while(self.DVS <= 2) :
            # record the data for today
            todays_data = pd.DataFrame([[self.today, self.DVS, self.LAI, self.TAGP, 
                                        self.storage, self.stem, self.leaves, self.roots]],
                                        columns=['Date', 'Development Stage', 'LAI', 'Biomass', 
                                               'Storage', 'Stem', 'Leaves', 'Roots'])
            self.data = pd.concat([self.data, todays_data], ignore_index=True, axis=0)

            self.update()
            
            total_radiation = float(self.weather.loc[self.today]['IRRAD']) 
            radiation_intercepted = total_radiation * (1 - np.exp(-1 * self.k * np.min([4, self.LAI])))
            if self.today == datetime.date(2006, 5, 9) :
                print('Total radiation ', total_radiation)
                print('radiation_intercepted ', radiation_intercepted)
                print(np.exp(-1 * self.k * np.min([4, self.LAI])), ' and then you 1 - that')
                print('roots part ', self.roots_partition)
            
            biomass = radiation_intercepted * self.RUE # later, potential biomass
            # actual biomass limited by water and nitrogen availability, but ignoring stress situations for now
            
            # then partition it
            self.TAGP += biomass * (1 - self.roots_partition)
            self.roots += biomass * self.roots_partition
            # update root depth here, but for now assuming water isn't a problem
            self.leaves += biomass * self.leaves_partition
            self.LAI += biomass * self.leaves_partition * self.LAI_growth_rate()
            self.three += 1
            #self.set_of_lv += biomass * self.leaves_partition
            if self.three >= 3 :
                dostuff = True
            
            self.stem += biomass * self.stem_partition
            self.storage += biomass * self.storage_partition

            # DVS = 1 is anthesis, DVS = 2 is maturity
            avg_temp = np.average([self.weather.loc[self.today]['TMIN'], self.weather.loc[self.today]['TMAX']])
            if self.DVS <= 1 :
                self.DVS += avg_temp / self.TSUM1
                self.DOA = self.today # Date of Anthesis
            else :
                self.DVS += avg_temp / self.TSUM2
            

In [160]:
# Data for the data :
# DVS, LAI, BIOMASS, TWSO, TW leaves stem & roots, 

# Data for summary: 
# LAIMAX, all those stuff above, Date of sowing, Emergence, anthesis, maturity, harvest

In [173]:
model = Model(crop_params, soil_params, weather_data, management_params)

In [174]:
model.run()

      0     1
0  0.00  0.60
1  0.33  0.58
2  0.40  0.55
3  0.80  0.10
4  1.00  0.00
5  2.00  0.00
Total radiation  23846.0
radiation_intercepted  21682.73968586067
0.09071795328941251  and then you 1 - that
roots part  0.55


In [176]:
model.data

,Date,Development Stage,LAI,Biomass,Storage,Stem,Leaves,Roots
0,2006-04-05,0,0.140000,0,0,0,0,0
1,2006-04-06,0.006125,26.555255,1200.69,0,0,1200.69,1801.04
2,2006-04-07,0.01475,247.241645,11231.9,0,0,11231.9,16847.8
3,2006-04-08,0.0225625,413.708638,18798.6,0,0,18798.6,28197.9
4,2006-04-09,0.0294375,839.577359,38156.2,0,0,38156.2,57234.4
...,...,...,...,...,...,...,...,...
114,2006-07-28,1.92038,24046.677778,4.01361e+06,1.8265e+06,1.09409e+06,1.09302e+06,1.2604e+06
115,2006-07-29,1.94081,24046.677778,4.05761e+06,1.85949e+06,1.10509e+06,1.09302e+06,1.2604e+06
116,2006-07-30,1.96208,24046.677778,4.0884e+06,1.88259e+06,1.11279e+06,1.09302e+06,1.2604e+06
117,2006-07-31,1.98106,24046.677778,4.13438e+06,1.91707e+06,1.12428e+06,1.09302e+06,1.2604e+06


In [ ]:
# Nitrogen - apply a standard amount at first and then apply a set amount to an area when it's Nitrogen starved
# Irrigation - maintain a certain pF